In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/generation_config.json


In [15]:
#For accessing the model
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [23]:
#For fine tuning the model
!capture
!pip install -U peft
!pip install -U trl

/bin/bash: line 1: capture: command not found


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

In [24]:
#Modules for fine tuning the model
from transformers import HfArgumentParser,TrainingArguments,logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, wandb
from datasets import load_dataset
from trl import SFTTrainer

In [8]:
#Quantization

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [9]:
#Accessing mistral model
model_name = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
#Pipeline function to access the model
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer = tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

Device set to use cuda:0


In [14]:
prompt = "You are a food expert, help me with a Healthy Pizaa receipe?"

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=200, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a food expert, help me with a Healthy Pizaa receipe?

## Healthy Pizza

Hi,

I was wondering if you could help me with a healthy pizza receipe?

I am not a food expert so I don't really know where to start.

I know I am suppose to use whole wheat dough, but what else?

Should I use low fat cheese?

I would like to make it taste good and still be healthy.

Thanks,

Joyce

## Healthy Pizza

Hi Joyce,

We have a great healthy pizza recipe that is perfect for you.

Here is what you need to do:

1. In a large bowl, combine 1/2 cup warm water (110°F/45°C), 2-1/2 tsp. active dry yeast, and 1 tsp. honey. Let stand for 5 minutes.

2. Meanwhile, combine


In [33]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_Key")
secret_wandb = user_secrets.get_secret("wandb")

In [35]:
#Hugging Face API to save and push the model to the Hugging Face Hub
!huggingface-cli login --token $secret_hf

usage: huggingface-cli <command> [<args>] login [-h] [--token TOKEN] [--add-to-git-credential]
huggingface-cli <command> [<args>] login: error: argument --token: expected one argument


In [36]:
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning mistral 7B', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: neerajmehta15 (neerajmehta15-stanza-living). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [39]:
base_model = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "mistral_7b_guanaco"

In [43]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="train")
dataset["text"][100]

'<s>[INST] cuanto es 2x2 xD [/INST] La respuesta es 4. </s><s>[INST] puedes demostrarme matematicamente que 2x2 es 4? [/INST] En una multiplicación, el producto es el resultado de sumar un factor tantas veces como indique el otro, es decir, si tenemos una operación v · n = x, entonces x será igual a v sumado n veces o n sumado v veces, por ejemplo, para la multiplicación 3 · 4 podemos sumar "3 + 3 + 3 + 3" o "4 + 4 + 4" y en ambos casos nos daría como resultado 12, para el caso de 2 · 2 al ser iguales los dos factores el producto sería "2 + 2" que es igual a 4 </s>'

In [44]:
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

In [46]:
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

In [47]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [50]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

In [57]:
# Ensure tokenizer padding side is set to right
tokenizer.padding_side = "right"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [59]:
tokenizer.pad_token = tokenizer.eos_token 

trainer = SFTTrainer(
    model=model, #Change model name
    train_dataset=dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [60]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.033100
50,1.475100
75,1.072900
100,1.344000
125,1.049200
150,1.243200
175,1.039200
200,1.281000
225,1.000800
250,1.326200


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=250, training_loss=1.186475112915039, metrics={'train_runtime': 6473.9244, 'train_samples_per_second': 0.154, 'train_steps_per_second': 0.039, 'total_flos': 1.8706482621874176e+16, 'train_loss': 1.186475112915039, 'epoch': 1.0})

In [61]:
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True

train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,▂▇▁▆▁▅▁█▁▄
train/learning_rate,▁▁▁▁▁▁▁▁▁▁
train/loss,▁█▂▆▂▅▂▅▁▆
total_flos,1.8706482621874176e+16
train/epoch,1
train/global_step,250
train/grad_norm,0.82715
train/learning_rate,0.0002
train/loss,1.3262


In [63]:
logging.set_verbosity(logging.CRITICAL)

prompt = "What is LLM fine tuning"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is LLM fine tuning [/INST] LLM fine tuning is a process of adapting a large language model (LLM) to a specific task or domain. This is done by training the model on a smaller dataset that is relevant to the task or domain. Fine tuning can improve the performance of the LLM on the specific task or domain, while preserving its general language understanding capabilities. 🤗 🤖 🤗 🤖 🤗 🤖 🤗 🤖 🤗 🤖 🤗 🤖 🤗 🤖 🤗 🤖 🤗 🤖 🤗 🤖 🤗 🤖


In [65]:
prompt = "How can I lean LLM fine tuning ?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How can I lean LLM fine tuning ? [/INST] There are a few ways to fine tune a language model. One way is to use a pre-trained model as a starting point and then fine tune it on a smaller dataset. Another way is to use a pre-trained model as a starting point and then fine tune it on a larger dataset. Finally, you can also use a pre-trained model as a starting point and then fine tune it on a dataset that is similar to the task you are trying to solve. 

Which of these methods you choose will depend on the specific task you are trying to solve and the resources you have available. 

If you have a small dataset, you may want to use a pre-trained model as a starting point and then fine tune it on a smaller dataset. If you have a large dataset, you may want to use a pre-trained model as a starting point and then fine tune
